In [ ]:
import yaml
from pathlib import Path
import pandas as pd
import numpy as np
from aicsimageio import AICSImage
import sys
sys.path.append("../../")
import segmentation_core as sc
import seg_eval as se
import seg_processing as sp
import pipeline_utils as pu
import utils
import importlib

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# project path 
c = yaml.safe_load(open("../../config.yaml", "r"))
PATH_PROJECT = Path(c['path_project'])
# data path
c = yaml.safe_load(open("./config.yaml", "r"))
PATH_DATA = Path(c['path_data'])
PATH_FNAMES = PATH_DATA / "fname-lookup-timelapse.csv"
PATH_SEG_FUNCS = PATH_PROJECT / "seg-func-map.yaml"
PATH_EVALS = Path("./eval-pdfs")

## choose the images to segment 
# idxs = ['WT-1-1','WT-2-4','D0-1-6','D0-3-2','D14-1-4','D14-2-9']
# idxs for WT and D0
# idxs = ['WT-2-5','WT-3-2','D0-1-6','D0-1-3','D0-2-6','D0-3-5']
# idxs = ['D7-2-1','D7-2-2','D7-3-7', 'D7-3-20']
# idxs = ['D14-1-9','D14-2-7','D14-3-10']
# idxs = ['D21-2-2','D21-3-6','D21-3-12']
idxs = ["AP24-ARNA-1","AP24-ARNA-11","AP24-CTRLAC-8"]
# idxs = ["AP48-ARNA-11","AP48-RNAOAC-6","AP48-CTRL-5","AP48-CTRLOAC-8"]
channel="mito"
frame=50


In [ ]:
df_fnames[df_fnames['g1']=="AP48"]
df_fnames.loc["WT-1-1"]["path_file"]

#### load img data

In [ ]:
# read in files and print their fnames 
df_fnames = pd.read_csv(PATH_FNAMES).set_index('index')
df_fnames['path_file'] = str(PATH_DATA) +"/"+ df_fnames['folder']+"/"+df_fnames['fname']
fnames = list(df_fnames.loc[idxs]['path_file'])
for i in range(len(fnames)):
    print(f"{idxs[i]}: {fnames[i]}")


In [ ]:
x.shape

In [ ]:
# x = img_obj.shape

In [ ]:
imgs = [] 
frame=0
zslice_mult=0.5
for fname in fnames: 
    img_obj = AICSImage(fname)
    zdim = img_obj.shape[2]
    zslice = int(zdim*zslice_mult)
    print(f"Shape {img_obj.shape}, getting frame {frame}, ch_idx {ch_idx}, zslice {zslice}")
    x = img_obj.data
    img=x[frame, :,[zslice]].copy()
    del x
    imgs.append(img)

In [ ]:
for img in imgs:
    plt.figure(figsize=(15,15))
    plt.imshow(img[0,2], cmap='gray')

In [ ]:
imgs[0].shape

#### segmentation info

In [ ]:
# load seg info 
## load segmentation functions
seg_funcs = sc.load_segmentation_functions(fname=PATH_SEG_FUNCS)

## segmentation config file 
config_fname =  "./configs/example-config.yaml"
seg_config = sc.load_segmentation_config_from_file(fname=config_fname, seg_funcs=seg_funcs, verbose=0)


In [ ]:
import yaml
dct = yaml.safe_load("""
mito:
    seg: 
        pipeline: 
            AC: 
                scaling_param: [0,19]
            #G2:
            #    sigma: 0.7
            ES:
                niter: 3
                kappa: 1.2
            or_branch:
                branch1:
                    S2:
                        s2_param: [[2,0.15], [3.3,0.15]]  
                branch2:
                    F2:
                        f2_param: [[1.15,0.55]] 
                        #f2_param: [[1.0,0.50],[3.3,0.35]] 

            S:
                min_size: 20
        ops: 
            inplace: False
""")
seg_config[channel] = dct[channel]


In [ ]:
importlib.reload(sp)
importlib.reload(sc)
frame=0
all_figs=[]
for i, fname in enumerate(fnames):
    ### record info about this file 
    idx=idxs[i]
    msg = "*"*80 + f"\nIdx: {idx}    channel: {channel}    frame: {frame}\n" + "*"*80
    print(msg)
    print(fnames[i])
    
    ### get the whole-cell mask 
    img_tmp = imgs[i].copy()
    f, (img_sum, img_sum_blurred, mask, mask_cell) = sp.whole_cell_segment_eval(fname, img_tmp, title=idx, merge_channels=['lyso','mito','golgi','peroxy','er',])
    print("Central cell mask")
    display(f)
    
    ## indexn only this channel take the 
    ch_idx_lookup = pu.get_channel_idx_lookup(img_obj.channel_names)
    ch_idx = ch_idx_lookup[channel]
    img_in = imgs[i][:,ch_idx]
    img = img_in.copy()

    ### do the segmentation
    f = plt.figure(figsize=(10,1)); plt.axis('off')
    plt.text(0.5,0.5,msg,ha='center',va='center', fontsize='xx-large', fontstretch='expanded')
    all_figs.append(f)
    plt.close()
    
    
    show_whole_seg, show_sliding_window_masks, show_sliding_window = True, False, True
    ## Warning - this is only for display. 
    ## The actual AC norm used in segmentation is the one listed in the YAML config file
    ac_norm_params=[[0,15]]   
    gs_smooth_params = [1,2]
    es_smooth_params = [
        dict(niter=5,kappa=1.2,gamma=0.1),
    #     dict(niter=40,kappa=1.2,gamma=0.1)
                       ]
    # do the segmentation 
    seg, imgs_inter = sc.seg_2d(img[0], seg_config, seg_funcs, ch_name=channel, inplace=False, save_inters=True, mask=mask_cell[0])

    # show whole image
    if show_whole_seg:
        f, axs = plt.subplots(1,2,figsize=(20,20))
        axs[0].imshow(img[0], cmap='gray')
        axs[1].imshow(seg[0], cmap='gray')
        plt.close()
        display(f)
        all_figs.append(f)

    # sliding window params
    window_sz, stride = 200, 200
    min_pixels, figsize_mult, max_display_cols, do_plot_dividing_lines = 10, 7, 4, 5
    f, show_mask, noshow_mask = pu.compare_seg_windows(img, seg, imgs_inter=imgs_inter,
                                            max_display_cols=max_display_cols, figsize_mult=figsize_mult,do_plot_dividing_lines=do_plot_dividing_lines,
                                            ac_norm_params=ac_norm_params,gs_smooth_params=gs_smooth_params, 
                                                      es_smooth_params=es_smooth_params)
    plt.close()
    display(f)
    all_figs.append(f)
    assert np.array_equal(img, img_in), "Warning: the image got modified inplace somewhere"

    if show_sliding_window_masks:
        f_tmp, axs = plt.subplots(1,2, figsize=(20,20))
        axs[0].imshow(noshow_mask, cmap='gray');    axs[0].set(title="Ignored windows")
        axs[1].imshow(show_mask, cmap='gray')  ;    axs[1].set(title="Included widnows")
        plt.show()



In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
fname_eval = PATH_EVALS / f"{channel}-4samples.pdf"
with PdfPages(fname_eval) as pdf:
    for f in all_figs:
        pdf.savefig(f)
    